In [1]:
from tlspt.datamodules.components.numpy_dataset import NumpyDataset
from tlspt.datamodules.components.base_site import BaseSiteDataset
from tlspt.datamodules.components.octree_dataset import OctreeDataset
from tlspt.datamodules.components.merged_dataset import MergedOctreeDataset
from tlspt.transforms import TLSSampler, UniformDownsample
from tlspt.datamodules.single_datamodule import SingleDataModule

from torch.utils.data import DataLoader

from tlspt.datamodules.components.octree_dataset import OctreeDatasetHdf5
from tlspt.transforms import Padder, HDF5ChunkSampler

from matplotlib import pyplot as plt

from loguru import logger

import sys
import numpy as np
import time

logger.remove()  # Remove the default handler
logger.add(sys.stdout, level="INFO")

1

In [2]:
plots_keep = ['SPA01', 'SPA02', 'SPA03', 'SPA04', 'SPA05', 'SPA06']
feature_names = None
features_to_normalize = None
num_points = 8192
batch_size = 128

In [ ]:
ply_dataset_disk = OctreeDataset(split_file='../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                                 split='train',
                                 feature_names=feature_names,
                                 features_to_normalize=features_to_normalize,
                                 scale=1.5,
                                 transform=UniformDownsample(num_points=num_points, replace="resample_as_req"),
                                 plots_keep=plots_keep)
ply_dataset_disk.prepare_data(force_compute=False)
ply_dataloader_disk = DataLoader(ply_dataset_disk, num_workers=0, batch_size=batch_size, shuffle=True)

In [ ]:
ply_dataset_memory = OctreeDataset(split_file='../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                                 split='train',
                                 feature_names=feature_names,
                                 features_to_normalize=features_to_normalize,
                                 scale=1.5,
                                 transform=UniformDownsample(num_points=num_points, replace="resample_as_req"),
                                 plots_keep=plots_keep,
                                 in_memory=True)
ply_dataset_memory.prepare_data(force_compute=False)
ply_dataloader_memory = DataLoader(ply_dataset_disk, num_workers=0, batch_size=batch_size, shuffle=True)

In [3]:
hdf5_dataset_disk = OctreeDatasetHdf5(split_file='../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                                 split='train',
                                 feature_names=feature_names,
                                 features_to_normalize=features_to_normalize,
                                 scale=1.5,
                                 idx_sampler=HDF5ChunkSampler(num_points=num_points),
                                 transform=Padder(num_points=num_points),
                                 plots_keep=plots_keep)
hdf5_dataset_disk.prepare_data(force_compute=False)
hdf5_dataloader_disk = DataLoader(hdf5_dataset_disk, num_workers=0, batch_size=batch_size, shuffle=True)

2024-11-23 22:08:12.747 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDatasetHdf5(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): reading splits from ../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv
2024-11-23 22:08:12.761 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDatasetHdf5(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): looking for 14 folders in ../data/plot_octrees/allen-spain/octrees/
2024-11-23 22:08:12.763 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDatasetHdf5(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): found 14 plots for 'train' out of 14 plots defined in split file
2024-11-23 22:08:12.765 | INFO     | tlspt.structures.file_octree:__init__:155 - Initializing oct

In [ ]:
%%time 
for i, batch in enumerate(ply_dataloader_disk):
    print(batch['points'].shape)
    if i > 10:
        break
    pass

In [4]:
%%time 
for i, batch in enumerate(hdf5_dataloader_disk):
    print(batch['points'].shape)
    if i > 10:
        break
    pass

torch.Size([128, 8192, 3])
torch.Size([128, 8192, 3])
torch.Size([128, 8192, 3])
torch.Size([128, 8192, 3])
torch.Size([128, 8192, 3])
torch.Size([128, 8192, 3])
torch.Size([128, 8192, 3])
torch.Size([128, 8192, 3])
torch.Size([128, 8192, 3])
torch.Size([128, 8192, 3])
torch.Size([128, 8192, 3])
torch.Size([128, 8192, 3])
CPU times: user 4min 37s, sys: 16.6 s, total: 4min 54s
Wall time: 50.9 s


In [ ]:
%%time 
for i, batch in enumerate(ply_dataloader_memory):
    print(batch['points'].shape)
    if i > 10:
        break
    pass

In [ ]:
print(ply_dataloader_disk.dataset.in_memory)

In [ ]:
print(ply_dataloader_memory.dataset.in_memory)